In [99]:
# pip install google-generativeai
import os
import gradio as gr
from dotenv import load_dotenv
import google.generativeai as palm
import re
import requests
import json
from textwrap import dedent
from langchain.tools import tool
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from crewai import Agent, Task
from crewai import Crew
from crewai.crews.crew_output import CrewOutput

In [185]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_PALM_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
YELP_API_KEY = os.getenv("YELP_API_KEY")
# Configure the API
palm.configure(api_key= GOOGLE_API_KEY)
# Yelp API endpoint
YELP_SEARCH_URL = "https://api.yelp.com/v3/businesses/search"
OPENAI_MODEL = "gpt-3.5-turbo"

In [186]:
def generate_text(prompt):
    completion = palm.generate_text(
        model="models/text-bison-001",
        prompt=prompt,
        temperature=0.7,
        max_output_tokens=800,
    )
    return completion.result

In [187]:
def parse_response(response):
    # Use regular expressions to find the sections more flexibly
    caviar_selection_match = re.search(r'Caviar Selection:\s*(.*)', response)
    champagne_pairing_match = re.search(r'Champagne Pairing:\s*(.*)', response)
    exclusive_insights_match = re.search(r'Exclusive Insights:\s*([\s\S]*)', response)

    caviar_selection = caviar_selection_match.group(1) if caviar_selection_match else None
    champagne_pairing = champagne_pairing_match.group(1) if champagne_pairing_match else None

    exclusive_insights = []
    if exclusive_insights_match:
        insights_section = exclusive_insights_match.group(1).strip()
        exclusive_insights = [line.strip().replace('*', '').strip() for line in insights_section.splitlines() if line.strip().startswith('*')]

    return caviar_selection, champagne_pairing, exclusive_insights

def caviar_coach(taste_profile, texture_preference, budget):
    image_url = "https://s.yimg.com/ny/api/res/1.2/.11X0wngxtZy7WtNp0r.2w--/YXBwaWQ9aGlnaGxhbmRlcjt3PTEyMDA7aD02NzQ-/https://media.zenfs.com/en/tasting_table_543/13b649b5330e192063fa2d41441150f3"
    prompt = f"""
    As a luxury caviar expert, provide a personalized recommendation based on the following preferences:
    Taste Profile: {taste_profile}
    Texture Preference: {texture_preference}
    Budget: ${budget}

    Please provide:
    1. A specific caviar recommendation
    2. A champagne pairing suggestion
    3. Some exclusive insights or tips for enjoying this caviar experience

    Format your response as follows:
    Caviar Selection: [Your recommendation]
    Champagne Pairing: [Your pairing suggestion]
    Exclusive Insights: [Your insights or tips]
    """

    try:
        # Get the generated response from the model
        response = generate_text(prompt)
        
        # Parse the response into sections using regular expressions
        caviar_selection, champagne_pairing, exclusive_insights = parse_response(response)
        
        # Format the response properly
        formatted_response = f"<div style='font-family: Arial, sans-serif; line-height: 1.6;'>"
        formatted_response += "<strong>Your Imperial Highness,</strong><br><br>"
        formatted_response += "Based on your exquisite preferences, I am honored to present our recommendation:<br><br>"
        caviar_selection.replace("**","")
        
        if caviar_selection:
            formatted_response += f"Caviar Selection: {caviar_selection}<br>"
        else:
            formatted_response += "<strong>Caviar Selection:</strong> Not available<br>"
        
        if champagne_pairing:
            formatted_response += f"Champagne Pairing: {champagne_pairing}<br><br>"
        else:
            formatted_response += "<strong>Champagne Pairing:</strong> Not available<br><br>"
        
        if exclusive_insights:
            formatted_response += "<strong>Exclusive Insights:</strong><br>"
            insights_html = "<br>".join([f"• {insight}" for insight in exclusive_insights if insight])  # Remove empty bullet points
            formatted_response += f"{insights_html}<br><br>"
        else:
            formatted_response += "<strong>Exclusive Insights:</strong> Not available<br><br>"
        # Add image to the response
        formatted_response += f"<img src='{image_url}' alt='Caviar Image' style='max-width: 50%; height: auto;'><br><br>"
        formatted_response += "May this experience elevate your senses and leave an indelible mark on your palate.<br><br>"
        formatted_response += "Bon appétit, Your Highness.<br><br>"
        formatted_response += "For more exclusive insights, visit us at: <a href='https://www.caviart.club' target='_blank'>here</a></div>"
        youtube_video_html = """<iframe width="560" height="315" src="https://www.youtube.com/embed/0QrUq7XWfvw" 
                             title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; 
                             encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>"""
        return formatted_response,youtube_video_html
    except Exception as e:
        return f"An error occurred: {str(e)}. Please check if the API key is set correctly."


In [202]:
class RestaurantSearchAgent:
    
    def __init__(self, api_key):
        self.api_key = api_key

    def search_restaurants(self, city, term="caviar"):
        headers = {
            "Authorization": f"Bearer {self.api_key}"
        }
        params = {
            "location": city,
            "term": term,
            "limit": 10  # Limiting results to top 10
        }
        response = requests.get(YELP_SEARCH_URL, headers=headers, params=params)
        
        if response.status_code == 200:
            return response.json()
        else:
            return None

# Task for finding restaurants with caviar in the provided city
class SearchRestaurantsTask:
    def __init__(self, agent, city):
        self.agent = agent
        self.city = city
    
    def run(self):
        result = self.agent.search_restaurants(self.city)
        if result and "businesses" in result:
            businesses = result["businesses"]
            output_html = "<div style='font-family:Arial;'>"
            output_html += "<h3>Here are the places to enjoy caviar:</h3>"
            for business in businesses:
                name = business.get("name", "Unknown Restaurant")
                address = ", ".join(business.get("location", {}).get("display_address", ["Unknown Address"]))
                rating = business.get("rating", "N/A")
                image_url = business.get("image_url", "")
                review_count = business.get("review_count", "0")
                url = business.get("url", "#")

                # Add HTML for each business
                output_html += f"""
                    <div style='border: 1px solid #ccc; padding: 10px; margin-bottom: 10px;'>
                        <img src="{image_url}" alt="{name}" style='width:150px; height:100px; float:left; margin-right:10px;'>
                        <h4><a href="{url}" target="_blank">{name}</a></h4>
                        <p><strong>Address:</strong> {address}</p>
                        <p><strong>Rating:</strong> {rating} / 5 ({review_count} reviews)</p>
                        <div style='clear:both;'></div>
                    </div>
                """
            output_html += "</div>"
            return output_html
        else:
            return "<p>No restaurants found in the specified city.</p>"

# Crew to run the agent and task
class RestaurantCrew:
    def __init__(self, city):
        self.city = city
        self.agent = RestaurantSearchAgent(YELP_API_KEY)
        self.task = SearchRestaurantsTask(self.agent, self.city)

    def run(self):
        return self.task.run()

In [203]:
def on_click(city):
  restaurant_crew = RestaurantCrew(city)
  try:
    result = restaurant_crew.run()
    return result
  except Exception as e:
    print(f"An error occurred: {e}")
    return "Something went wrong..."

In [204]:
def clear_recommendation():
    return ""

In [205]:
# Gradio interface for caviar coach function and restaurant finder
with gr.Blocks(title="🍾 Imperial Caviar Connoisseur 🥂") as demo:

    # Section for the caviar coach functionality
    with gr.Column():
        # User preferences for caviar coach
        preferred_taste = gr.Dropdown(["Mild & Delicate", "Rich & Buttery", "Bold & Intense"], label="Preferred Taste Profile")
        preferred_texture = gr.Dropdown(["Soft & Creamy", "Firm & Distinct"], label="Preferred Texture")
        budget = gr.Slider(minimum=50, maximum=1000, step=50, label="Budget (USD)")
        
        # Recommendation output
        recommendation_output = gr.HTML(label="Your Royal Recommendation")

        # Buttons in a row below the budget slider
        with gr.Row():
            submit_button = gr.Button("Get Caviar Recommendation")
            clear_button = gr.Button("Clear Results")

    # When submit_button is clicked, trigger caviar_coach
    submit_button.click(
        fn=caviar_coach,
        inputs=[preferred_taste, preferred_texture, budget], # Use a hidden textbox for city
        outputs=[recommendation_output]
    )

    # When clear_button is clicked, clear the recommendation output
    clear_button.click(
        fn=clear_recommendation,
        inputs=[],
        outputs=[recommendation_output]
    )

    # Section for finding caviar restaurants
    with gr.Column():
        # City input for finding restaurants
        city_input = gr.Textbox(label="Enter your city to find caviar places")
         # Buttons in a row below the city input
        with gr.Row():
            find_caviar_button = gr.Button("Find Caviar Places")
            clear_places_button = gr.Button("Clear Results")

        # Restaurant search output
        restaurant_output = gr.HTML(label="")

    # When find_caviar_button is clicked, trigger on_click for restaurant search
    find_caviar_button.click(
        fn=on_click,
        inputs=[city_input],
        outputs=[restaurant_output]
    )

    clear_places_button.click(
        fn=clear_recommendation,
        inputs=[],
        outputs=[restaurant_output]
    )

In [206]:
if __name__ == "__main__":
   demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7895
Running on public URL: https://b0e6d8130c5749d70b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
